# Visualizing the Hydrogel-Cell Interaction Solution

In this notebook, we visualize the hydrogel-cell interaction solution from the optimization algorithm. First we load the necessary libraries in the first input cell, then in the second cell there is the settings of the simulation.

In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse as spm
from scipy.io import loadmat, savemat

import matplotlib.pyplot as plt
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
import sys, time, os.path
from itertools import permutations

module_path = '../module/'
sys.path.append(module_path)
from SHUtil import SphCoord_to_CartCoord, CartCoord_to_SphCoord

import pyshtools
from SHUtil import SHCilmToVector, SHVectorToCilm, lmk2K, K2lmk
from SHUtil import plotfv, TransMat, l_coeffs, m_coeffs, LM_list
from ShElastic import calSmode, calUmode
from SHBV import generate_submat, visualize_Cmat, print_SH_mode

from scipy.sparse.linalg import lsqr, spsolve
from scipy.interpolate import RectBivariateSpline
from scipy.optimize import minimize

from Case05_utilities import d2v, d2e, d2f
from Case05_utilities import Uvec2Tvec, Tvec2Uvec
from Case05_utilities import SHVec2mesh, visSHVec, visSH3d
from Case05_utilities import SHvec_rtoc, SHvec_ctor
from Case05_utilities import generate_fcache, generate_Xneigh, generate_Eneigh, generate_Fneigh

## The following cell is the setting of the simulation

`shapename` is the number of the shape.
`smoothed` indicates whether the shape is smoothed or not.
`dilated` indicates the traction-free mask, `_smoothed` for smoothing of the edge, `_1um_dilated` for dilation of the edge.

In [ ]:
############################# change inputs here #################################
# Data file name
datadir = '../testdata'
smoothed = 'smoothed' #lmax20_lwork15'
dilated = '_softedge' # dilated = '_1um_dilated'; # dilated = '';
shapename = 'Shape2'
datafile = os.path.join(datadir, shapename+'_Coordinates_Cart_'+smoothed+'.csv')
connfile = os.path.join(datadir, shapename+'_Connectivity.csv')
maskfile = os.path.join(datadir, shapename+'_Mask'+dilated+'.csv')

# Material properties
mu0 = 300/3; nu0 = 0.499;
# Spherical Harmonics Analysis Settings
lJmax = 20; lKmax = 20; lmax_plot = 60;

# initial guess settings
# AKinitfile = 'AK_iter_'+shapename+('_lmax%d'%20)+'_1um_dilated.npy'
# ldamp_lo = int(lJmax/4); ldamp_hi = lJmax;

lwork = 15

# regularizations
myalpha = 1e-3  # traction magnitude
mybeta  = 1e-4  # coefficient magnitude

# program switches
plot_figure = True
node_face_dist_on = None
opt_proc = [2, ]; myord = 1; # 2: sol2dr; myord: p-norm order
#opt_proc = [0, 1, 0, 1, 0, 1, 0, 1, 0]  # 1: node-face on; 0: node-face off;

# minimization settings
N_period = 2
maxiter_per_period = 1
CG_gtol = 1e-4
minimizer = 'CG'
minimizer_config = {'maxiter': maxiter_per_period, 'gtol': CG_gtol, 'disp': True}
# method='Powell', options={'maxiter':1, 'maxfev':10, 'disp': True})

# dump files for minimization
savefig = False
savename = shapename+('_a%.0e_b%.0e'%(myalpha, mybeta))+('_lmax%d'%lJmax)+smoothed+dilated
AKfile = 'AK_iter_'+savename+'.npy'
fvfile = 'fv_'+savename+'.txt'

# settings for loading \hat{U}, \hat{T} coefficients
shtype = 'reg'
coeff_dir = os.path.join(module_path, 'lmax%dmodes'%60)

# neibor list settings
# n_list = 200
# n_load = 10
# n_recal_neigh = 1
# file_neigh = shapename + '_neigh%d_lmax%d'%(n_list, lJmax) + smoothed + dilated

#################################################################################
mu = 1.; nu = nu0;
Umodes = loadmat(os.path.join(coeff_dir,'Umodes.mat'))
Umodes = (Umodes['U1'+shtype], Umodes['U0'+shtype])
fullDmat = calUmode(Umodes, mu, nu)
Dmat = generate_submat(mu, nu, fullDmat, lKmax, lJmax, kJ=3).tocsc()

Tmodes = loadmat(os.path.join(coeff_dir,'Tmodes.mat'))
Tmodes = (Tmodes['T1'+shtype], Tmodes['T2'+shtype], Tmodes['T3'+shtype], Tmodes['T0'+shtype])
fullCmat = calSmode(Tmodes, mu, nu)
Cmat = generate_submat(mu, nu, fullCmat, lKmax, lJmax, kJ=3).tocsc()

In [ ]:
#### load the geometry ####
# Vs, Vp: list of nodes (nV, trivial), coordinates of the nodes (nVx3)
# Es, Ep: list of edges (nEx2), list of points on the edges (nEx2x3)
# Fs, Fp: list of facets (nFx3), list of points on the facets (nFx3x3)
# Tfv: traction free boundary map of the node list
# Tfe: traction free boundary map of the edge list
# Tff: traction free boundary map of the face list
# Tf_diluted: diluted traction free boundary map

data = np.genfromtxt(datafile, delimiter=',')
conn = np.genfromtxt(connfile, delimiter=',', dtype=np.int)
if dilated == '_softedge':
    masktype = np.float
else:
    masktype = np.int
if shapename == 'Shape4':
    mask = np.zeros_like(data[:,0]).astype(masktype)
else:
    mask = np.genfromtxt(maskfile, dtype=masktype)
print('data, connectivity:', data.shape, conn.shape)

Fs = conn - 1
Np = data.shape[0]
Vs = np.arange(Np)
edge_conn = spm.lil_matrix((Np, Np), dtype=bool)
for i, j in permutations(range(3), 2):
    edge_conn[Fs[:, i], Fs[:, j]] = True
Es = spm.triu(edge_conn).tocoo()
Es = np.vstack([Es.row, Es.col]).T
print('id of nodes, edges, facets:', Vs.shape, Es.shape, Fs.shape)
Vp = data[..., :3]
Ep = Vp[Es, :]; Fp = Vp[Fs, :];
print('coord of nodes, edges, facets:', Vp.shape, Ep.shape, Fp.shape)

if dilated == '_softedge':
    Tfv = (mask > 0.5)
else:
    Tfv = mask.astype(np.bool)

#### Plot the geometry (Vp) ####
fig = plt.figure()#figsize=(16,16))
ax = fig.add_subplot(111, projection='3d')

nTfv = np.logical_not(Tfv)
ax.scatter3D(Vp[Tfv, 0], Vp[Tfv, 1], Vp[Tfv, 2])
ax.scatter3D(Vp[nTfv, 0], Vp[nTfv, 1], Vp[nTfv, 2])

ax.view_init(azim=0, elev=0)

ax.set_aspect('equal')
plt.show()

Then we determine the original radius of the particle, assuming the particle is incompressible. The total volume can be estimated by adding the volume of the tetrahedrons. The volume of a tetrahedron is calculated as:

$$
V_{0123}=\frac{1}{6}
\begin{vmatrix}
 x_1 & y_1 & z_1 & 1\\ 
 x_2 & y_2 & z_2 & 1\\ 
 x_3 & y_3 & z_3 & 1\\ 
 0 & 0 & 0 & 1\\ 
\end{vmatrix}
$$

In [ ]:
tet = np.zeros((Fs.shape[0], 4, 4))
tet[:,:-1,:-1] = Fp
tet[:,:,-1] = 1
vol = np.sum(np.linalg.det(tet)/6, axis=0)
r0 = np.cbrt(vol/(4/3*np.pi))
print('V = %.4f, r0 = %.4f'%(vol, r0))

The next cell shows the convergence behavior of the optimization. The first subplot includes the curve for all three parts of the target function: average shape-difference $\Delta x$, average traction magnitude $\alpha\|T\|$ and the regularization term $\beta\|p_Ka_K\|$, where $p_K = p_{(k,l,m)} = \exp(l-l_{work}) - 1,\; (l_{work}\le l\le l_{\max})$.

In [ ]:
plot_funval = np.loadtxt(fvfile)[:, 1:]
plt.figure(figsize=(14,5))
plt.subplot(131)
plt.plot(plot_funval[:, 0], label=r'total')
if plot_funval.shape[1] > 5:
    plt.plot(plot_funval[:, 3], label=r'$\Delta x$')
    plt.plot(plot_funval[:, 4]*myalpha, label=r'$\alpha\|T\|$')
    plt.plot(plot_funval[:, 5]*mybeta, label=r'$\beta\|p_Ka_K\|$')
plt.legend()
plt.title('target')
plt.xlabel(r'$n_{period}$')
plt.subplot(132)
plt.plot(plot_funval[:, 1], label='Converged $\Delta x = %.4f$nm'%(plot_funval[-1, 1]*1e3))
plt.title('shape_dist $\Delta x(\mu m)$')
plt.xlabel(r'$n_{period}$')
plt.legend()
plt.subplot(133)
plt.plot(plot_funval[:, 2], label='Converged $\|T\| = %.4f$Pa'%(plot_funval[-1, 2]))
plt.title('traction(Pa)')
plt.xlabel(r'$n_{period}$')
plt.legend()
if savefig:
    plt.savefig(savename+'_convergence.png', transparent=True)
plt.show()

In the next cell, we plot the spherical harmonic coefficients of displacement $\hat{U}_K$, traction $\hat{T}_K$, and the coefficients in the solution space $a_K$.

In [ ]:
# Load the solution file
AK_sol = np.load(AKfile)
Uvec1 = Dmat.dot(AK_sol); Tvec1 = Cmat.dot(AK_sol)

print('Plot the Solution Coefficients')
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.plot(np.abs(Uvec1)); plt.ylim(0, 0.01); plt.title('$\hat{U}_{lmk}$')
plt.subplot(132)
plt.plot(np.abs(AK_sol)); plt.ylim(0, 0.01); plt.title('$a_K$')
plt.subplot(133)
plt.plot(np.abs(Tvec1)); plt.ylim(0, 0.1); plt.title('$\hat{T}_{lmk}$')
if savefig:
    plt.savefig(savename+'_coeffs.png', transparent=True)
plt.show()

In the next cell, we plot the displacement map and traction map correspond to the solution.

In [ ]:
print('Plot displacement map')
fig, ax = visSHVec(Uvec1*r0, lmax_plot=lmax_plot,
                   SphCoord=True, Complex=True,     # spherical coord switch, complex coefficient switch
                   # n_vrange=(-1,1), s_vrange=(0,0.01), # normal and shear map value range
                   config_quiver=(2, 3, 'k', 20),   # setting of quiver in shear map
                   lonshift=180,                    # shift the map in longitude direction
                   figsize=(12,6), show=False)       # figure size
if savefig:
    fig[0].savefig(savename+'_un.png', transparent=False)
    fig[1].savefig(savename+'_us.png', transparent=False)
plt.show()

print('Plot traction map')
fig, ax = visSHVec(Tvec1*r0, lmax_plot=lmax_plot, 
                   SphCoord=True, Complex=True,     # spherical coord switch, complex coefficient switch
                   # n_vrange=(-200, 200), s_vrange=(0,0.01), # normal and shear map value range
                   config_quiver=(2, 3, 'k', 100), # setting of quiver in shear map
                   lonshift=180,                    # shift the map in longitude direction
                   figsize=(12,6), show=False)       # figure size
if savefig:
    fig[0].savefig(savename+'_Tn.png', transparent=False)
    fig[1].savefig(savename+'_Ts.png', transparent=False)
plt.show()

In the next cell, we compare the shape using the radial displacement $u_r$. The fitted shape has radial displacement of $u_t$, and the difference between the data and the fitted shape is also given.

In [ ]:
# Vp -> ur (radial displacement) -> u1 (x,y,z)
Vr, Vthe, Vphi = CartCoord_to_SphCoord(Vp[...,0], Vp[...,1], Vp[...,2])
Vphi[Vphi < 0] += 2*np.pi
Vlat = 90-np.rad2deg(Vthe)
Vlon = np.rad2deg(Vphi)
ur = (Vr - r0)/r0

urcilm_interp, chi2ur = pyshtools.expand.SHExpandLSQ(ur, Vlat, Vlon, lmax=lJmax)
ucoeff_interp = pyshtools.SHCoeffs.from_array(urcilm_interp)
urgrid_interp = ucoeff_interp.expand('GLQ')

latsdeg, lonsdeg = pyshtools.expand.GLQGridCoord(lmax_plot)
lon = np.deg2rad(lonsdeg)
colat = np.deg2rad(90-latsdeg)
PHI, THETA = np.meshgrid(lon, colat)
R = np.ones_like(PHI)
X,Y,Z = SphCoord_to_CartCoord(R, THETA, PHI)
X0 = np.stack([X,Y,Z], axis=-1)
Xt = X0 + SHVec2mesh(Uvec1, lmax=lmax_plot, SphCoord=False, Complex=True)

rt, thet, phit = CartCoord_to_SphCoord(Xt[...,0], Xt[...,1], Xt[...,2])
phit[phit < 0] += 2*np.pi
latt = 90-np.rad2deg(thet)
lont = np.rad2deg(phit)
urt = rt - 1

rtcilm_interp, chi2ut = pyshtools.expand.SHExpandLSQ(urt.flatten(), latt.flatten(), lont.flatten(), lmax=lJmax)
print('Residuals for expanding into spherical harmonics. ur: %.4f ut: %.4f'%(chi2ur, chi2ut))

In [ ]:
utcoeff_interp = pyshtools.SHCoeffs.from_array(rtcilm_interp)
utgrid_interp = utcoeff_interp.expand('GLQ')
fig, ax = plotfv(urgrid_interp.to_array()*r0, lonshift=180, show=False, figsize=(6,3))
ax.set_title('$u_r (\mu m)$')
if savefig:
    fig.savefig(savename+'_ur.png', transparent=True)
fig, ax = plotfv(utgrid_interp.to_array()*r0, lonshift=180, show=False, figsize=(6,3))
ax.set_title('$u_t (\mu m)$')
if savefig:
    fig.savefig(savename+'_ut.png', transparent=True)

udgrid_interp = urgrid_interp.copy()
TT = np.deg2rad(90 - udgrid_interp.lats())
PP = np.deg2rad(udgrid_interp.lons())
ud_tmp = utgrid_interp.to_array() - urgrid_interp.to_array()
udz_tmp = ud_tmp * np.cos(TT[:,np.newaxis])
ud_new = np.sqrt(ud_tmp**2 - udz_tmp**2 + (udz_tmp)**2)
udgrid_interp.data = np.sign(ud_tmp) * ud_new #* np.sin(TT[:,np.newaxis])
fig, ax = plotfv(udgrid_interp.to_array()*r0, lonshift=180, show=False, cmap='Reds', figsize=(6,3))
ax.set_title('$\Delta u = u_r-u_t (\mu m)$')
if savefig:
    fig.savefig(savename+'_ud.png', transparent=True)
plt.show()

dist_ud = np.mean((udgrid_interp.data**2)*np.sin(TT[:,np.newaxis]))
print('Average squared distance on unit sphere:', dist_ud)
print('Averaged shape difference %.8f(um)'%(np.sqrt(dist_ud)*r0))
plt.hist(udgrid_interp.data.flatten()*r0, bins=100)
plt.title('distribution of the shape difference')
plt.ylabel('# of nodes'); plt.xlabel('$\Delta u(\mu m)$');
if savefig:
    plt.savefig(savename+'_dhist.png', transparent=True)
plt.show()

In [ ]:
print('Evaluate Displacement and Traction in Cartesian Coordinates for plotting 3d shape...')
umesh1 = SHVec2mesh(Uvec1, lmax=lJmax, SphCoord=False, Complex=True)
tmesh1 = SHVec2mesh(Tvec1*mu0, lmax=lJmax, SphCoord=False, Complex=True)
umesh1_fine = SHVec2mesh(Uvec1, lmax=lmax_plot, SphCoord=False, Complex=True)
tmesh1_fine = SHVec2mesh(Tvec1*mu0, lmax=lmax_plot, SphCoord=False, Complex=True)

In [ ]:
print('Visualize the nodes in 3D...')
fig, ax = visSH3d(umesh1, cmesh=tmesh1, r0=r0, show=False, elevation=0, azimuth=0, surface=False, figsize=(4,4))
nTfv = np.logical_not(Tfv)
ax.scatter3D(Vp[Tfv, 0], Vp[Tfv, 1], Vp[Tfv, 2], marker='^', s=10)
ax.scatter3D(Vp[nTfv, 0], Vp[nTfv, 1], Vp[nTfv, 2], marker='^', s=10)
if savefig:
    plt.tight_layout(); plt.savefig(savename+'_compare.png', transparent=True)
plt.show()

In [ ]:
print('Visualize the shape in 3D...')
fig, ax = visSH3d(umesh1_fine, cmesh=tmesh1_fine, 
                  r0=r0, show=False, 
                  elevation=0, azimuth=0, 
                  surface=True, figsize=(4,4))

# reference data
ax.scatter3D(Vp[Tfv, 0], Vp[Tfv, 1], Vp[Tfv, 2], marker='^', s=10)
ax.scatter3D(Vp[nTfv, 0], Vp[nTfv, 1], Vp[nTfv, 2], marker='^', s=10)
ax.set_title('Front view')
if savefig:
    plt.tight_layout(); plt.savefig(savename+'_front.png', transparent=True)
plt.show()

In [ ]:
fig, ax = visSH3d(umesh1_fine, cmesh=tmesh1_fine, 
                  r0=r0, show=False, 
                  elevation=0, azimuth=-90, 
                  surface=True, figsize=(4,4))

# Plot reference data
ax.scatter3D(Vp[Tfv, 0], Vp[Tfv, 1], Vp[Tfv, 2], marker='^', s=10)
ax.scatter3D(Vp[nTfv, 0], Vp[nTfv, 1], Vp[nTfv, 2], marker='^', s=10)
ax.set_title('Left view')
if savefig:
    plt.tight_layout(); plt.savefig(savename+'_left.png', transparent=True)
plt.show()

In [ ]:
fig, ax = visSH3d(umesh1_fine, cmesh=tmesh1_fine, 
                  r0=r0, show=False, 
                  elevation=90, azimuth=0, 
                  surface=True, figsize=(4,4))

# Plot reference data
ax.scatter3D(Vp[Tfv, 0], Vp[Tfv, 1], Vp[Tfv, 2], marker='^', s=10)
ax.scatter3D(Vp[nTfv, 0], Vp[nTfv, 1], Vp[nTfv, 2], marker='^', s=10)
ax.set_title('Top view')
if savefig:
    plt.tight_layout(); plt.savefig(savename+'_top.png', transparent=True)
plt.show()

In [ ]:
umesh_Cart = SHVec2mesh(Uvec1*r0, lmax=lmax_plot, SphCoord=False, Complex=True)
tmesh_Cart = SHVec2mesh(Tvec1*mu0, lmax=lmax_plot, SphCoord=False, Complex=True)
umesh_Sph = SHVec2mesh(Uvec1*r0, lmax=lmax_plot, SphCoord=True, Complex=True)
tmesh_Sph = SHVec2mesh(Tvec1*mu0, lmax=lmax_plot, SphCoord=True, Complex=True)
ugrid_Cart = SHVec2mesh(Uvec1*r0, lat=Vlat, lon=Vlon, lmax=lmax_plot, SphCoord=False, Complex=True)
tgrid_Cart = SHVec2mesh(Tvec1*mu0, lat=Vlat, lon=Vlon, lmax=lmax_plot, SphCoord=False, Complex=True)
ugrid_Sph = SHVec2mesh(Uvec1*r0, lat=Vlat, lon=Vlon, lmax=lmax_plot, SphCoord=True, Complex=True)
tgrid_Sph = SHVec2mesh(Tvec1*mu0, lat=Vlat, lon=Vlon, lmax=lmax_plot, SphCoord=True, Complex=True)
savemat(savename+'.mat', {'umesh_Cart':umesh_Cart, 'tmesh_Cart':tmesh_Cart,
                          'umesh_Sph': umesh_Sph,  'tmesh_Sph': tmesh_Sph,
                          'ugrid_Cart':ugrid_Cart, 'tmesh_Cart':tgrid_Cart,
                          'ugrid_Sph': ugrid_Sph,  'tmesh_Sph': tgrid_Sph,
                          'lat': Vlat, 'lon': Vlon, 'r0': r0,  'mu0': mu0})